In [1]:
# #turn off gpu support, no need for it
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
#Import external functions
from image_processing import *
from utils import *
from custom_models import *
from inception_utils import *

from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, TensorBoard, EarlyStopping

import os

Using TensorFlow backend.


In [3]:
# Load images for training
path_to_images = 'data_generator/UCF-101-frames'

target_size = (240,320)
batch_size = 32

train_folder = path_to_images + '/train'
train_datagen = ImageDataGenerator(rotation_range=10,
                                   preprocessing_function=preprocess_input,
                                   rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.33)
training_batches2 = train_datagen.flow_from_directory(train_folder,
                                                     target_size=target_size,
                                                     batch_size=batch_size,
                                                     interpolation='bicubic',
                                                     subset='training')
training_batches = crop_generator(training_batches2, crop_width, crop_height, random=True)

validation_batches2 = train_datagen.flow_from_directory(train_folder,
                                                       target_size=target_size,
                                                       batch_size=batch_size,
                                                       interpolation='bicubic',
                                                       subset='validation')
validation_batches = crop_generator(validation_batches2, crop_width, crop_height, random=True)

test_folder = path_to_images + '/test'
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255)
testing_batches2 = test_datagen.flow_from_directory(test_folder,
                                                   target_size=target_size,
                                                   batch_size=batch_size,
                                                   interpolation='bicubic')
testing_batches = crop_generator(testing_batches2, crop_width, crop_height, random=False)

classes_dictionary = {}
for cls, idx in training_batches2.class_indices.items():
    classes_dictionary[idx] = cls

Found 956573 images belonging to 101 classes.
Found 471071 images belonging to 101 classes.
Found 540949 images belonging to 101 classes.


In [4]:
# Single frame model definition
input_shape = (crop_height,crop_width,3)

#SGD Optimizer Parameters
base_lr = 0.0001
momentum = 0.9

In [5]:
model = get_model(weights='imagenet',input_shape=input_shape,num_labels=num_labels)
model = freeze_all_but_top(model)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 113, 113, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 113, 113, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
model.fit_generator(training_batches, 
                    steps_per_epoch=100,
                    validation_data=validation_batches,
                    validation_steps=10,
                    epochs=10,
                    callbacks=[])

Epoch 1/10
100/100 [==============================] - 40s 396ms/step - loss: 4.2893 - acc: 0.1231 - val_loss: 4.4801 - val_acc: 0.1469
Epoch 2/10
100/100 [==============================] - 35s 347ms/step - loss: 3.0927 - acc: 0.2900 - val_loss: 3.7545 - val_acc: 0.2625
Epoch 3/10
100/100 [==============================] - 35s 352ms/step - loss: 2.5751 - acc: 0.3922 - val_loss: 3.8362 - val_acc: 0.2562
Epoch 4/10
100/100 [==============================] - 35s 352ms/step - loss: 2.2061 - acc: 0.4537 - val_loss: 3.1145 - val_acc: 0.3656
Epoch 5/10
100/100 [==============================] - 35s 350ms/step - loss: 2.0424 - acc: 0.5000 - val_loss: 3.9570 - val_acc: 0.2938
Epoch 6/10
100/100 [==============================] - 35s 350ms/step - loss: 1.8551 - acc: 0.5247 - val_loss: 3.6032 - val_acc: 0.3563
Epoch 7/10
100/100 [==============================] - 35s 348ms/step - loss: 1.7186 - acc: 0.5613 - val_loss: 3.2651 - val_acc: 0.3563
Epoch 8/10
100/100 [==============================] - 3

In [7]:
filepath = 'saved_models/inception_rgb.hdf5'
checkpointer = ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True)
# early_stopper = EarlyStopping(patience=10)
# tensorboard = TensorBoard(log_dir=os.path.join('data', 'logs-inception'))

model = freeze_all_but_mid_and_top(model=model,base_lr=base_lr,momentum=momentum)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 113, 113, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 113, 113, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [8]:
model.fit_generator(training_batches,
                    steps_per_epoch=100,
                    validation_data=validation_batches,
                    validation_steps=10,
                    epochs=1000,
                    callbacks=[checkpointer])

Epoch 1/1000
100/100 [==============================] - 43s 433ms/step - loss: 1.3648 - acc: 0.6406 - top_k_categorical_accuracy: 0.8634 - val_loss: 2.6623 - val_acc: 0.3875 - val_top_k_categorical_accuracy: 0.6625

Epoch 00001: val_loss improved from inf to 2.66230, saving model to saved_models/inception_rgb.hdf5
Epoch 2/1000
100/100 [==============================] - 36s 365ms/step - loss: 1.1764 - acc: 0.6887 - top_k_categorical_accuracy: 0.8897 - val_loss: 2.5377 - val_acc: 0.4031 - val_top_k_categorical_accuracy: 0.6937

Epoch 00002: val_loss improved from 2.66230 to 2.53765, saving model to saved_models/inception_rgb.hdf5
Epoch 3/1000
100/100 [==============================] - 36s 365ms/step - loss: 1.0414 - acc: 0.7250 - top_k_categorical_accuracy: 0.9069 - val_loss: 2.5188 - val_acc: 0.4469 - val_top_k_categorical_accuracy: 0.7063

Epoch 00003: val_loss improved from 2.53765 to 2.51877, saving model to saved_models/inception_rgb.hdf5
Epoch 4/1000
100/100 [======================

100/100 [==============================] - 36s 364ms/step - loss: 0.5532 - acc: 0.8431 - top_k_categorical_accuracy: 0.9666 - val_loss: 2.3666 - val_acc: 0.4875 - val_top_k_categorical_accuracy: 0.7063

Epoch 00030: val_loss did not improve from 1.97590
Epoch 31/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.5777 - acc: 0.8441 - top_k_categorical_accuracy: 0.9622 - val_loss: 1.9542 - val_acc: 0.5375 - val_top_k_categorical_accuracy: 0.7781

Epoch 00031: val_loss improved from 1.97590 to 1.95419, saving model to saved_models/inception_rgb.hdf5
Epoch 32/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.5544 - acc: 0.8512 - top_k_categorical_accuracy: 0.9656 - val_loss: 2.0270 - val_acc: 0.5563 - val_top_k_categorical_accuracy: 0.7812

Epoch 00032: val_loss did not improve from 1.95419
Epoch 33/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.5647 - acc: 0.8475 - top_k_categorical_accuracy: 0.9663 - val_loss: 2.1711 

100/100 [==============================] - 36s 363ms/step - loss: 0.3296 - acc: 0.9100 - top_k_categorical_accuracy: 0.9887 - val_loss: 1.9014 - val_acc: 0.5594 - val_top_k_categorical_accuracy: 0.8031

Epoch 00090: val_loss did not improve from 1.81330
Epoch 91/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.3217 - acc: 0.9137 - top_k_categorical_accuracy: 0.9794 - val_loss: 2.3488 - val_acc: 0.5219 - val_top_k_categorical_accuracy: 0.7156

Epoch 00091: val_loss did not improve from 1.81330
Epoch 92/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.3146 - acc: 0.9184 - top_k_categorical_accuracy: 0.9847 - val_loss: 2.0145 - val_acc: 0.5594 - val_top_k_categorical_accuracy: 0.8031

Epoch 00092: val_loss did not improve from 1.81330
Epoch 93/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.3245 - acc: 0.9134 - top_k_categorical_accuracy: 0.9850 - val_loss: 2.4737 - val_acc: 0.4938 - val_top_k_categorical_accuracy: 0


Epoch 00150: val_loss did not improve from 1.70423
Epoch 151/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.2146 - acc: 0.9416 - top_k_categorical_accuracy: 0.9928 - val_loss: 1.9746 - val_acc: 0.5625 - val_top_k_categorical_accuracy: 0.8063

Epoch 00151: val_loss did not improve from 1.70423
Epoch 152/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.2273 - acc: 0.9391 - top_k_categorical_accuracy: 0.9916 - val_loss: 2.3193 - val_acc: 0.5344 - val_top_k_categorical_accuracy: 0.7500

Epoch 00152: val_loss did not improve from 1.70423
Epoch 153/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.1924 - acc: 0.9456 - top_k_categorical_accuracy: 0.9938 - val_loss: 2.0195 - val_acc: 0.5750 - val_top_k_categorical_accuracy: 0.8000

Epoch 00153: val_loss did not improve from 1.70423
Epoch 154/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.2067 - acc: 0.9503 - top_k_categorical_accuracy: 0.9919 - 


Epoch 00211: val_loss did not improve from 1.70423
Epoch 212/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.1548 - acc: 0.9634 - top_k_categorical_accuracy: 0.9950 - val_loss: 2.0428 - val_acc: 0.6094 - val_top_k_categorical_accuracy: 0.7844

Epoch 00212: val_loss did not improve from 1.70423
Epoch 213/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.1613 - acc: 0.9575 - top_k_categorical_accuracy: 0.9938 - val_loss: 1.8380 - val_acc: 0.6188 - val_top_k_categorical_accuracy: 0.8250

Epoch 00213: val_loss did not improve from 1.70423
Epoch 214/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.1664 - acc: 0.9569 - top_k_categorical_accuracy: 0.9944 - val_loss: 2.0371 - val_acc: 0.5875 - val_top_k_categorical_accuracy: 0.8281

Epoch 00214: val_loss did not improve from 1.70423
Epoch 215/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.1495 - acc: 0.9591 - top_k_categorical_accuracy: 0.9963 - 

...


Epoch 00760: val_loss did not improve from 1.65346
Epoch 761/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.0387 - acc: 0.9891 - top_k_categorical_accuracy: 1.0000 - val_loss: 2.1549 - val_acc: 0.5969 - val_top_k_categorical_accuracy: 0.8313

Epoch 00761: val_loss did not improve from 1.65346
Epoch 762/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0408 - acc: 0.9875 - top_k_categorical_accuracy: 0.9997 - val_loss: 1.8674 - val_acc: 0.6312 - val_top_k_categorical_accuracy: 0.8281

Epoch 00762: val_loss did not improve from 1.65346
Epoch 763/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0365 - acc: 0.9903 - top_k_categorical_accuracy: 0.9994 - val_loss: 2.0205 - val_acc: 0.6156 - val_top_k_categorical_accuracy: 0.8125

Epoch 00763: val_loss did not improve from 1.65346
Epoch 764/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0438 - acc: 0.9887 - top_k_categorical_accuracy: 0.9991 - 


Epoch 00821: val_loss did not improve from 1.65346
Epoch 822/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0407 - acc: 0.9909 - top_k_categorical_accuracy: 0.9991 - val_loss: 2.1295 - val_acc: 0.5875 - val_top_k_categorical_accuracy: 0.7937

Epoch 00822: val_loss did not improve from 1.65346
Epoch 823/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0433 - acc: 0.9884 - top_k_categorical_accuracy: 0.9991 - val_loss: 2.0381 - val_acc: 0.6219 - val_top_k_categorical_accuracy: 0.8156

Epoch 00823: val_loss did not improve from 1.65346
Epoch 824/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.0448 - acc: 0.9872 - top_k_categorical_accuracy: 0.9991 - val_loss: 2.1427 - val_acc: 0.6094 - val_top_k_categorical_accuracy: 0.8187

Epoch 00824: val_loss did not improve from 1.65346
Epoch 825/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.0361 - acc: 0.9922 - top_k_categorical_accuracy: 0.9997 - 

100/100 [==============================] - 36s 363ms/step - loss: 0.0403 - acc: 0.9906 - top_k_categorical_accuracy: 0.9997 - val_loss: 2.3113 - val_acc: 0.5969 - val_top_k_categorical_accuracy: 0.7781

Epoch 00882: val_loss did not improve from 1.64329
Epoch 883/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0358 - acc: 0.9912 - top_k_categorical_accuracy: 0.9994 - val_loss: 1.8945 - val_acc: 0.6094 - val_top_k_categorical_accuracy: 0.8313

Epoch 00883: val_loss did not improve from 1.64329
Epoch 884/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0416 - acc: 0.9881 - top_k_categorical_accuracy: 0.9994 - val_loss: 2.1255 - val_acc: 0.6062 - val_top_k_categorical_accuracy: 0.8063

Epoch 00884: val_loss did not improve from 1.64329
Epoch 885/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0350 - acc: 0.9900 - top_k_categorical_accuracy: 0.9994 - val_loss: 2.4063 - val_acc: 0.5813 - val_top_k_categorical_accuracy


Epoch 00912: val_loss did not improve from 1.64329
Epoch 913/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0310 - acc: 0.9909 - top_k_categorical_accuracy: 1.0000 - val_loss: 2.0897 - val_acc: 0.6000 - val_top_k_categorical_accuracy: 0.8531

Epoch 00913: val_loss did not improve from 1.64329
Epoch 914/1000
100/100 [==============================] - 36s 364ms/step - loss: 0.0438 - acc: 0.9891 - top_k_categorical_accuracy: 0.9991 - val_loss: 2.1625 - val_acc: 0.5938 - val_top_k_categorical_accuracy: 0.7969

Epoch 00914: val_loss did not improve from 1.64329
Epoch 915/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0301 - acc: 0.9953 - top_k_categorical_accuracy: 1.0000 - val_loss: 2.1297 - val_acc: 0.6438 - val_top_k_categorical_accuracy: 0.8156

Epoch 00915: val_loss did not improve from 1.64329
Epoch 916/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0359 - acc: 0.9897 - top_k_categorical_accuracy: 0.9994 - 


Epoch 00973: val_loss did not improve from 1.64329
Epoch 974/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0378 - acc: 0.9900 - top_k_categorical_accuracy: 0.9994 - val_loss: 2.4216 - val_acc: 0.5750 - val_top_k_categorical_accuracy: 0.8000

Epoch 00974: val_loss did not improve from 1.64329
Epoch 975/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0356 - acc: 0.9900 - top_k_categorical_accuracy: 0.9997 - val_loss: 2.0383 - val_acc: 0.5906 - val_top_k_categorical_accuracy: 0.8187

Epoch 00975: val_loss did not improve from 1.64329
Epoch 976/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0396 - acc: 0.9887 - top_k_categorical_accuracy: 0.9988 - val_loss: 2.2805 - val_acc: 0.6219 - val_top_k_categorical_accuracy: 0.7969

Epoch 00976: val_loss did not improve from 1.64329
Epoch 977/1000
100/100 [==============================] - 36s 363ms/step - loss: 0.0384 - acc: 0.9891 - top_k_categorical_accuracy: 0.9991 - 

In [9]:
scores = model.evaluate_generator(generator=testing_batches,verbose=1,steps=testing_batches2.samples//batch_size)
print(scores)


16904/16904 [==============================] - 3614s 214ms/step
[1.7998858336006656, 0.6354949272361571, 0.8436501715570279]
